<div class='alert-block alert-info'>
    <br>
    <h1 align="center"> AI Project 
    <h3 align="center">Nadia Godje </h3>
    <h3 align="center">Stive </h3>
    <\br>
</div>

Pour commencer, nous devons charger les données des défauts à partir du fichier CSV "defects.csv" et définir les informations sur les biscuits

In [65]:
import pandas as pd

# Chargement des données des défauts depuis le fichier CSV
def load_defects_data():
    defects_data = pd.read_csv('defects.csv')
    return defects_data

defects_data = load_defects_data()

biscuits_info = [
    {'num':0,'length': 4, 'value': 6, 'defauts': {'a': 4, 'b': 2, 'c': 3}},  # Biscuit 0
    {'num':1,'length': 8, 'value': 12, 'defauts': {'a': 5, 'b': 4, 'c': 4}},  # Biscuit 1
    {'num':2,'length': 2, 'value': 1, 'defauts': {'a': 1, 'b': 2, 'c': 1}},  # Biscuit 2
    {'num':3,'length': 5, 'value': 8, 'defauts': {'a': 2, 'b': 3, 'c': 2}}   # Biscuit 3
]

roll_length = 500

print(biscuits_info)
print(roll_length)
print(defects_data.head()) 


[{'num': 0, 'length': 4, 'value': 6, 'defauts': {'a': 4, 'b': 2, 'c': 3}}, {'num': 1, 'length': 8, 'value': 12, 'defauts': {'a': 5, 'b': 4, 'c': 4}}, {'num': 2, 'length': 2, 'value': 1, 'defauts': {'a': 1, 'b': 2, 'c': 1}}, {'num': 3, 'length': 5, 'value': 8, 'defauts': {'a': 2, 'b': 3, 'c': 2}}]
500
            x class
0  355.449335     c
1   92.496236     a
2  141.876795     c
3  431.833902     c
4  435.028461     c


Ce code charge les informations sur les biscuits et la longueur du rouleau, puis charge les données des défauts à partir du fichier CSV. Il affiche également les lignes des données des défauts que les biscuits peuvent supporter

Après avoir chargé les données des défauts et défini les informations sur les biscuits, nous pouvons commencer à formuler l'algorithme pour placer les biscuits sur le rouleau tout en respectant les contraintes spécifiées. 

**Greedy-search**

La fonction find_valid_positions cherche pour chaque biscuit la meilleure position possible sur le rouleau en considérant les contraintes de chevauchement et les seuils de défauts. Elle itère sur chaque position possible pour un biscuit, vérifie s'il y a un chevauchement avec d'autres biscuits déjà placés et si le seuil de défauts est respecté.

Puis, la fonction calculate_total_value calcule la **valeur totale des biscuits placés** (ce qu'on cherche à optimiser).

In [66]:
def find_valid_positions(biscuits_info, roll_length, defects_data):
    
    valid_positions = []
    for i, biscuit in enumerate(biscuits_info):
        # Recherche de la meilleure position pour chaque biscuit
        best_positions = []
        best_value = 0
        
        for position in range(roll_length - biscuit['length'] + 1):
            # Vérification de chevauchement
            overlap = False
            for placed_biscuit in valid_positions:
                if position <= placed_biscuit['position'] < position + biscuit['length']:
                    overlap = True
                    break
            
            if not overlap:
                # Vérification des défauts dans l'intervalle du biscuit
                defects_in_interval = defects_data[(defects_data['x'] >= position) &
                                                  (defects_data['x'] < position + biscuit['length'])]
                
                # Vérification des défauts par classe pour le biscuit actuel
                valid_biscuit = True
                for class_name, threshold in biscuit['defauts'].items():
                    class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
                    if class_defects.shape[0] > threshold:
                        valid_biscuit = False
                        break
                
                if valid_biscuit:
                    total_value = biscuit['value']
                    if best_positions:
                        total_value += best_value
                    if total_value > best_value:
                        best_value = total_value
                        best_positions = [position]
                    elif total_value == best_value:
                        best_positions.append(position)
    
        if best_positions:
            for pos in best_positions:
                valid_positions.append({'biscuit_': i, 'position': pos})
    
    return valid_positions

def calculate_total_value(valid_positions, biscuits_info):
    total_value = 0
    
    for pos in valid_positions:
        biscuit_id = pos['biscuit_']
        total_value += biscuits_info[biscuit_id]['value']
    
    return total_value

# Utilisation de la fonction pour trouver les positions valides pour placer les biscuits
valid_positions = find_valid_positions(biscuits_info, roll_length, defects_data)
print(valid_positions)

# Utilisation de la fonction pour calculer la valeur totale des biscuits placés
total_biscuit_value = calculate_total_value(valid_positions, biscuits_info)
print(f"La valeur totale des biscuits placés est : {total_biscuit_value}")



[{'biscuit_': 0, 'position': 496}, {'biscuit_': 1, 'position': 488}, {'biscuit_': 2, 'position': 497}, {'biscuit_': 3, 'position': 491}]
La valeur totale des biscuits placés est : 27


In [ ]:
def place_biscuits(biscuits_info, roll_length, defects_data):
    placed_positions = []
    while True:
        new_positions = find_valid_positions(biscuits_info, roll_length, defects_data)
        if not new_positions:
            break
        placed_positions.extend(new_positions)
    return placed_positions

# Utilisation de la fonction pour placer les biscuits sur le rouleau
placed_biscuits = place_biscuits(biscuits_info, roll_length, defects_data)
print(placed_biscuits)


**CSP (Constraint Satisfaction Problem)**

Le CSP  est un bon moyen de modéliser ce problème pour trouver **les meilleures positions** pour placer les biscuits tout en **respectant les contraintes**. Pour cela, nous allons utiliser la **bibliothèque python-constraint**, qui permet de définir des variables, des domaines et des contraintes pour résoudre ce genre de problèmes.

In [68]:
from constraint import Problem

def intervals_overlap(start1, end1, start2, end2):# on vérifie si 2 intervalles se chevauchent
    return not (end1 <= start2 or start1 >= end2)

#fonction pour vérifier si un biscuit peut être placé sans chevauchement et dépassement du seuil de défauts
def can_place_biscuit(position, length, defects_data, biscuit_defauts):
    #vérifie les defauts dans l'intervalles où le biscuit peut être placé [position, position + length]
    defects_in_interval = defects_data[(defects_data['x'] >= position) &
                                      (defects_data['x'] < position + length)]

    for class_name, nombre_de_defauts_max in biscuit_defauts.items():
        class_defects = defects_in_interval[defects_in_interval['class'] == class_name]
        if class_defects.shape[0] > nombre_de_defauts_max:
            return False
    
    return True

def solve_biscuit_placement(biscuits_info, roll_length, defects_data):
    problem = Problem()

    for i, biscuit in enumerate(biscuits_info):
        problem.addVariable(f"biscuit_{i}_position", range(roll_length - biscuit['length'] + 1))

    for i, biscuit in enumerate(biscuits_info):
        biscuit_length = biscuit['length']
        biscuit_defaut = biscuit['defauts']
        for j, other_biscuit in enumerate(biscuits_info):
            if i != j:
                other_biscuit_length = other_biscuit['length']
                problem.addConstraint(
                    lambda pos1, pos2, len1=biscuit_length, len2=other_biscuit_length:
                    not intervals_overlap(pos1, pos1 + len1, pos2, pos2 + len2),
                    (f"biscuit_{i}_position", f"biscuit_{j}_position")
                )

        problem.addConstraint(
            lambda pos, length=biscuit_length, thresholds=biscuit_defaut:
            can_place_biscuit(pos, length, defects_data, thresholds),
            (f"biscuit_{i}_position",)
        )

    solutions = problem.getSolutions()
    return solutions

# utilisation de la fonction pour résoudre le problème de placement des biscuits
solutions = solve_biscuit_placement(biscuits_info, roll_length, defects_data)
print(solutions)
